In [ ]:
from __future__ import division, print_function

import numpy as np
import matplotlib.pyplot as plt

from corner import corner
from acor import acor
from UL_uncert import UL_uncert

%matplotlib inline

In [ ]:
params = [r'$\cos\theta$', r'$\phi$', r'$\psi$', r'$\log_{10} h$', r'$t_0$ (MJD)']
ch_dict = {}

In [ ]:
analysis = 'UL'
#analysis = 'Det'

## BayesEphem

In [ ]:
ephem = 'BayesEphem'

chaindir = '/home/pbaker/nanograv/bwm/{}/{}/'.format(analysis, ephem)
chain_raw = np.loadtxt(chaindir + 'chain_1.txt')
    
burn = int(0.1 * len(chain_raw))
chain = chain_raw[burn:]
chain_bwm = chain[:,-20:-15]  # with BayesEphem

# de-log x-axis for ULs
if analysis == 'UL':
    chain_bwm[:,-2] = 10**chain_bwm[:,-2]
    params[-2] = r'$h$'

corr = acor(chain_bwm[:,3])[0]
N = len(chain_bwm)
print("BayesEphem")
print("N = {}, corr = {}, Neff = {}".format(N, corr, N/corr))

ch_dict['BayesEphem'] = chain_bwm

## Per ephemeris runs

In [ ]:
ephems = ['DE421', 'DE430', 'DE435', 'DE436']

for eph in ephems:
    chaindir = '/home/pbaker/nanograv/bwm/{}/{}/'.format(analysis, eph)
    chain_raw = np.loadtxt(chaindir + 'chain_1.txt')
    
    burn = int(0.1 * len(chain_raw))
    chain = chain_raw[burn:]    
    chain_bwm = chain[:,-9:-4]  # no ephem model

    # de-log x-axis for ULs
    if analysis == 'UL':
        chain_bwm[:,-2] = 10**chain_bwm[:,-2]
        params[-2] = r'$h$'
    
    corr = acor(chain_bwm[:,3])[0]
    N = len(chain_bwm)
    print(eph)
    print("N = {}, corr = {}, Neff = {}".format(N, corr, N/corr))
    print("")
    
    ch_dict[eph] = chain_bwm

# Plots!

In [ ]:
corner_kwargs = {'bins':30,
                 'labels':params,
                 'smooth':1.0,
                 'plot_datapoints':True,
                 'plot_density':False,
                 'plot_contours':False,
                 'fill_contours':True,}

fig = corner(ch_dict['DE421'], color='C0', **corner_kwargs);
#corner(ch_dict['DE430'], color='C1', fig=fig);
#corner(ch_dict['DE435'], color='C2', fig=fig);
corner(ch_dict['DE436'], color='C3', fig=fig, **corner_kwargs);

In [ ]:
ax = []
fig = plt.figure(figsize=(20,12))

for pp, par in enumerate(params):
    ax.append(fig.add_subplot(2,3,pp+1))
    for eph in ephems:
        ax[pp].hist(ch_dict[eph][:,pp], 30, normed=True, histtype='step', label=eph)
    ax[pp].set_xlabel(par)

ax[0].legend(loc='upper left')
ax[3].set_yscale('log')
ax[4].set_yscale('log')

In [ ]:
corner_kwargs = {'bins':30,
                 'labels':params,
                 'smooth':1.0,
                 'plot_datapoints':True,
                 'plot_density':False,
                 'plot_contours':False,
                 'fill_contours':True,}

corner(ch_dict['BayesEphem'], **corner_kwargs);

In [ ]:
ax = []
colors = ['C0', 'C1', 'C2', 'C3', 'k']
runs = ['DE421', 'DE430', 'DE435', 'DE436', 'BayesEphem']
fig = plt.figure(figsize=(20,12))

for pp, par in enumerate(params):
    ax.append(fig.add_subplot(2,3,pp+1))
    for eph,col in zip(runs, colors):
        ax[pp].hist(ch_dict[eph][:,pp], 30, normed=True, histtype='step', label=eph, color=col)
    ax[pp].set_xlabel(par)


ax[0].legend(loc='upper left')
ax[3].set_yscale('log')
ax[4].set_yscale('log')

In [ ]:
if analysis == "UL":
    for run in runs:
        UL, dUL = UL_uncert(ch_dict[run][:,-2])
        print("{}: {:.2e} +/- {:.2e}".format(run, UL, dUL))